In [1]:
import pandas as pd
import numpy as np
import os

from itertools import product

import Finalized_pipeline

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
main_datasets = ["BACE_class", "BACE_regre", "ROR_class", "ROR_regre"]
model_name_shorts = {"DecisionTreeRegressor": "dt", "DecisionTreeClassifier": "dt", 
                    "RandomForestRegressor": "rf", "RandomForestClassifier": "rf", 
                    "LinearRegression": "lr", "LogisticRegression": "lr", 
                    "MLPRegressor": "nn", "MLPClassifier": "nn",  
                    "GradientBoostingRegressor": "gb", "GradientBoostingClassifier": "gb",
                    "XGBRegressor": "xg", "XGBClassifier": "xg", 
                    "SVR": "sv", "SVC": "sv"}
    
top_models = 10


In [3]:
for dataset in main_datasets:
    regression = False
    if "regre" in dataset:
        regression = True


    df = pd.read_csv(r"Merged runs/" + dataset + ".csv")
    
    setups = {"Fingerprints": [], "Descriptors": [], "DescFing": []}

    for row in df.head(top_models)[['hyperparams', 'preprocessing', 'model']].iterrows():
        #print(row)
        params = list(row[1])[0].split(", '")
        params[0] = params[0][2:]
        params[-1] = params[-1][:-1]
        #print(params)

        preprocessing = list(row[1])[1]
        model = list(row[1])[2]
        #print(preprocessing)

        clean_params = {}
        
        for param in params:
            name = param.split("': ")[0]
            val = param.split("': ")[1]
            if val[0] == "'":
                val = val[1:-1]
            elif val[0] == "(":
                if val == "(50, 50)": val = (50, 50)
                if val == "(100, )" or val == "(100,)": val = (100, )
                if val == "(50, )" or val == "(50,)": val = (50, )
            elif val == "None":
                val = None
            else:
                try:
                    val = int(val)
                except:
                    val = float(val)
            clean_params[name] = [val]
        #print(model)
        #print(preprocessing)
        #print(clean_params)
        setups[preprocessing].append([model, clean_params])

    for key in setups.keys():
        setups[key] = sorted(setups[key], key=lambda x:x[0])
    #print(setups)

    returned_df = pd.DataFrame(columns=df.columns)
    #print(returned_df)

    for key in setups.keys():
        if key == "Descriptors":
            calculate_descriptors = True
            calculate_fingerprints = False
        if name == "DescFing":
            calculate_descriptors = True
            calculate_fingerprints = True
        if key == "Fingerprints":
            calculate_descriptors = False
            calculate_fingerprints = True

            
        if "BACE" in dataset:
            prepared_dataset = pd.read_csv(r"C:\Users\wojci\Documents\GitHub\czasteczkowa-inzynierka\experiments\split_datasets\split0.9_bace.csv")
        else:
            prepared_dataset = pd.read_csv(r"C:\Users\wojci\Documents\GitHub\czasteczkowa-inzynierka\experiments\split_datasets\split0.9_ROR_data_1.csv")
        
        if not "pIC50" in prepared_dataset.columns:
            prepared_dataset = Finalized_pipeline.calculate_pIC50(prepared_dataset, "target")
        
        if not regression:
            prepared_dataset = Finalized_pipeline.calculate_classification_labels(prepared_dataset, "pIC50", threshold=7)
            target_column = "label"
        else:
            target_column = "pIC50"

        if "mol" in prepared_dataset.columns:
            prepared_dataset.rename(columns={"mol": "SMILES"}, inplace=True)

        if regression:
            runtype = "regression"
        else:
            runtype = "classification"
        
        prepared_dataset = Finalized_pipeline.calculate_features(prepared_dataset, calculate_descriptors=calculate_descriptors, 
                                                                 calculate_fingerprints=calculate_fingerprints, 
                                                    SMILES_column_name="SMILES", target_column_name=target_column, 
                                                    split_column_name="Split", output_csv_path=dataset + "_" + key + "_0.9.csv")

        for combination in setups[key]:
                            # model short name, parameters
            hyperparams = {str(model_name_shorts[combination[0]]): combination[1]}


            new_df = Finalized_pipeline.hyperparameter_search(prepared_dataset, hyperparams, unique=False, return_df=True, output_file_name="tmp.csv")

            new_df['split'] = 0.9
            new_df['preprocessing'] = key

            returned_df = pd.concat([returned_df, new_df])

    if "mse" in returned_df.columns:
        returned_df.sort_values(by=['mse'], inplace=True)
    else:
        returned_df.sort_values(by=['roc_auc'], inplace=True, ascending=False)
    returned_df.to_csv("RERUN_" + dataset + ".csv", index=False)

        
                



A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Regression:  False
gb
{'max_depth': 3, 'loss': 'exponential', 'n_estimators': 200, 'learning_rate': 1.0, 'min_impurity_decrease': 0.05}
time:  3.00487
0 0.8139413906710595 0.9895282375959022 0.8157894736842105 0.9897134459955915 0.835820895522388 0.988135593220339 0.7671232876712328 0.988135593220339 0.8 0.988135593220339 GradientBoostingClassifier {'max_depth': 3, 'loss': 'exponential', 'n_estimators': 200, 'learning_rate': 1.0, 'min_impurity_decrease': 0.05}
Regression:  False
nn
{'hidden_layer_sizes': (100,), 'activation': 'relu', 'alpha': 0.001, 'max_iter': 1000, 'solver': 'sgd'}


Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


time:  66.47983
0 0.8545170799375759 0.9618490184440194 0.8552631578947368 0.9610580455547392 0.8591549295774648 0.943801652892562 0.8356164383561644 0.9677966101694915 0.8472222222222222 0.9556485355648535 MLPClassifier {'hidden_layer_sizes': (100,), 'activation': 'relu', 'alpha': 0.001, 'max_iter': 1000, 'solver': 'sgd'}


overflow encountered in reduce                    
100%|██████████| 1513/1513 [04:11<00:00,  6.02it/s]


...

time:  4.313194
0 0.7322154471544716 0.8835379362751918 0.7351598173515982 0.8845958312150483 0.6938775510204082 0.8451380552220888 0.7083333333333334 0.8778054862842892 0.7010309278350516 0.8611620795107034 MLPClassifier {'hidden_layer_sizes': (50,), 'activation': 'logistic', 'alpha': 0.001, 'max_iter': 200, 'solver': 'lbfgs'}



A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Regression:  True
rf
{'max_depth': 20, 'min_samples_split': 10, 'min_samples_leaf': 1, 'min_impurity_decrease': 0, 'n_estimators': 10}
time:  0.842575
0 0.6908633368674165 0.4407845001603857 0.4772921502275815 0.19429097558164105 0.5354403615750455 0.34305076946011026 0.4709753710732597 0.7895237460009403 RandomForestRegressor {'max_depth': 20, 'min_samples_split': 10, 'min_samples_leaf': 1, 'min_impurity_decrease': 0, 'n_estimators': 10}
Regression:  True
sv
{'C': 0.01, 'kernel': 'poly', 'epsilon': 0.1, 'gamma': 0.1}
time:  3.431881
0 0.6437773563415514 0.3226326123516708 0.4144492845381169 0.10409180255286346 0.479852630961752 0.18929873357837215 0.5406296146769098 0.8872369001815509 SVR {'C': 0.01, 'kernel': 'poly', 'epsilon': 0.1, 'gamma': 0.1}
Regression:  True
sv
{'C': 0.1, 'kernel': 'poly', 'epsilon': 0.1, 'gamma': 0.05}
time:  3.610779
0 0.644751799043391 0.2924855987906634 0.4157048823696893 0.0855478254999329 0.4813292033833727 0.17170518703441479 0.5392379257991187 0.9073256


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 2186/2186 [07:04<00:00,  5.16it/s] 

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 2186/2186 [07:05<00:00,  5.13it/s] 


Regression:  True
gb
{'max_depth': 7, 'loss': 'huber', 'n_estimators': 100, 'learning_rate': 0.1, 'min_impurity_decrease': 0.2}
time:  101.678838
0 0.6375208169234879 0.25634766270736253 0.4064327920107914 0.0657141241755277 0.4723842972195164 0.17247839968849005 0.5495149943810183 0.9288116051201657 GradientBoostingRegressor {'max_depth': 7, 'loss': 'huber', 'n_estimators': 100, 'learning_rate': 0.1, 'min_impurity_decrease': 0.2}
